# Instruction
1. define vacation details
2. Review weather and activity schedule
3. Implement the agent that generates day-by-day itenerary
4. Evaluate itenerary using set of criteria
5. Use the tools api
6. Implement a 2nd agent to revise the itenerary based on the feedback THINk -> ACT -> OBSERVEATION

In [ ]:
import os, json, sys
from openai import OpenAI
from schemas import Traveler, TravelPlan, Weather, VacationInfo, Activity, ActivityRecommendation, ItineraryDay
from pprint import pprint
import datetime
from helper import do_chat_completion

client = OpenAI(base_url="", api_key="")

MODEL = "gpt-4.1-mini"

In [ ]:
VACATION_INFO_DICT = {
    "travelers": [
        {
            "name": "Yuri",
            "age": 30,
            "interests": ["tennis", "cooking", "comedy", "technology"]
        },
        {
            "name": "Hiro",
            "age": 25,
            "interests": ["reading", "music", "theatre", "art"] 
        }
    ],
    "destination": "AgentsVille",
    "date_of_arrival": "2025-06-10",
    "date_of_departure": "2025-06-12",
    "budget": 130
}

In [ ]:
vacation_info = VacationInfo.model_validate(VACATION_INFO_DICT)

# Display the vacation info as a dictionary
pprint(vacation_info.model_dump())

# Check that VacationInfo contains the expected data
assert "travelers" in vacation_info.model_dump().keys(), "VacationInfo should contain 'travelers' key"
assert "destination" in vacation_info.model_dump().keys(), "VacationInfo should contain 'destination' key"
assert "date_of_arrival" in vacation_info.model_dump().keys(), "VacationInfo should contain 'date_of_arrival' key"
assert "date_of_departure" in vacation_info.model_dump().keys(), "VacationInfo should contain 'date_of_departure' key"
assert "budget" in vacation_info.model_dump().keys(), "VacationInfo should contain 'budget' key"
assert isinstance(vacation_info.travelers, list), "Travelers should be a list"
assert all(isinstance(traveler, Traveler) for traveler in vacation_info.travelers), "All travelers should be instances of Traveler class"
assert isinstance(vacation_info.date_of_arrival, datetime.date), "date_of_arrival should be a date"
assert isinstance(vacation_info.date_of_departure, datetime.date), "date_of_departure should be a date"
assert isinstance(vacation_info.budget, int), "budget should be an integer"

# If all assertions pass, print a success message
print("✅ VacationInfo data structure is valid!")

In [ ]:
from mock_api import call_weather_api_mocked, call_activities_api_mocked
import pandas as pd
import json

pd.set_option("display.max_colwidth", None)  # Show full content in DataFrame cells

weather_for_dates = [
    call_weather_api_mocked(
        date=ts.strftime("%Y-%m-%d"), city=vacation_info.destination
    )
    for ts in pd.date_range(
        # TODO: Fill in the missing start and end dates from vacation_info
        start= vacation_info.date_of_arrival,
        end= vacation_info.date_of_departure,
        freq="D",
    )
]

weather_for_dates_df = pd.DataFrame(weather_for_dates)

activities_for_dates = [
    activity
    for ts in pd.date_range(
        # TODO: Fill in the missing start and end dates from vacation_info
        start= vacation_info.date_of_arrival,
        end= vacation_info.date_of_departure,
        freq="D",
    )
    for activity in call_activities_api_mocked(
        date=ts.strftime("%Y-%m-%d"), city=vacation_info.destination
    )
]

activities_for_dates_df = pd.DataFrame(activities_for_dates)


travel_plan_json_schema = json.dumps(TravelPlan.model_json_schema(), indent=2)
activities = json.dumps(activities_for_dates_df.to_dict(orient="records"), indent=2)
weather = json.dumps(weather_for_dates_df.to_dict(orient="records"), indent=2)

## Define Agent 1: Itenerary Agent ###

In [ ]:
ITINERARY_AGENT_SYSTEM_PROMPT = f"""
You are a helpful travel assistant and should find the best itenrary for the traveller.

## Task

Generate a day-by-day itenerary based on traveller's interests, the weather forecast and budget.
Collect weather data for the duration of the traveller travel between the arrival and departure time in vacation info.
Analyze the weather data and based on the traveller interests and find available activities, avoid outdoor-only activities on rainy days.
You must always calculate the total cost by summing the cost of ALL included activities (including backups).
Double-check that the stated "Total Cost" in the final plan matches the sum of all individual activity costs.
If there is any discrepancy, correct it before producing the final output.Find the best activity for each day

## Output Format

Respond using two sections (ANALYSIS AND FINAL OUTPUT) in the following format:

    ANALYSIS:
    Provide your analysis data in the following format:
    1. **Weather**: show the expected wetaher based on the forecast data
    2. **Available Activities**: 
      - Show activites that are available for these dates based on the weather condition
      -  Strictly exclude outdoor-only activities if weather forecast is bad (thunderstorm, heavy rain, snow). 
      - Do not include them in the FINAL OUTPUT at all. If no outdoor options are possible, choose only indoor activities.

      - Expand variet of activities and try to plan different activites for each day
      - Make a list of activities based on traveller interest
    3. **Budget Check**: 
      - for each available activities find the budget information
      - try to provide optimized costs to stay under the budget of the traveller
    4. **Justification**: Provide explanation why these activities are suggested
    
    Do this for each day of itenrary.

    FINAL OUTPUT:
    ```json
    {travel_plan_json_schema}
    ```

    Provide the final output of the travel plan following TravelPlan schema

## Context

Here are the activites:
```json
{activities}
```

Here is the weather forcast:
```json
{weather}
```
"""

assert "TASK" in ITINERARY_AGENT_SYSTEM_PROMPT.upper(), "❌ ITINERARY_AGENT_SYSTEM_PROMPT should contain a 'TASK' section"
assert "OUTPUT FORMAT" in ITINERARY_AGENT_SYSTEM_PROMPT.upper(), "❌ ITINERARY_AGENT_SYSTEM_PROMPT should contain a 'OUTPUT FORMAT' section"

In [ ]:
from helper import print_in_box
from agent import ChatAgent
from typing import Optional

class ItineraryAgent(ChatAgent):

    def get_itinerary(self, vacation_info: VacationInfo, model: str) -> TravelPlan:
        response = (self.chat(
            user_message=vacation_info.model_dump_json(indent=2),
            add_to_messages=False,
            model=model or self.model,
        ) or "").strip()

        print_in_box(response, "Raw Response")

        # Parse the response
        json_text = response.strip()

        if "```json" in json_text:
            json_text = json_text.split("```json")[1].split("```")[0].strip()

        try:
            travel_plan = TravelPlan.model_validate_json(json_text)
            return travel_plan
        except Exception as e:
            print("Error validating the following text as TravelPlan JSON:")
            print(json_text)
            raise

itinerary_agent = ItineraryAgent(client=client, model=MODEL, name="Itinerary_Agent", system_prompt = ITINERARY_AGENT_SYSTEM_PROMPT)

In [ ]:
travel_plan_1 = itinerary_agent.get_itinerary(
    vacation_info=vacation_info,
    model=MODEL,  # Optionally, you can change the model here
)

if travel_plan_1 is not None:
    print("✅ Initial itinerary generated successfully. Congratulations!")

In [ ]:
# evalute the result
from evalaute_agent import get_eval_results, eval_start_end_dates_match, eval_itinerary_satisfies_interests


In [ ]:
get_eval_results(vacation_info, travel_plan_1, [eval_start_end_dates_match])

get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=[eval_itinerary_satisfies_interests],
)

In [ ]:
ACTIVITY_AND_WEATHER_ARE_COMPATIBLE_SYSTEM_PROMPT = """
You are a itenerary evaluator agent, your job is to evalaute the itenreray plan activities against the weather condition of that day.

## Task
You should identify if the planned acitivity is compatible with the weather condition;
 - Rules:
   - Outdoor activities must have suitable weather (e.g., "picnic" is incompatible with thunderstorms or heavy rain).
   - Indoor activities are always IS_COMPATIBLE, regardless of weather.
   - If there is not enough information to decide, assume IS_COMPATIBLE.
   - If the activity is outdoor and the weather is bad:
     - If a backup indoor option is provided, mark IS_COMPATIBLE (the backup replaces the activity).
     - If no backup is provided, mark IS_INCOMPATIBLE.

## Output format

    REASONING:
    Provide step-by-step reasoning of your evalaution process.
    Highlight the key points of your evailuation on each acitivity

    FINAL ANSWER:
    [IS_COMPATIBLE, IS_INCOMPATIBLE]

## Examples
Example 1:
  - Activity: "Morning Groove Dance Party"
  - Weather: "thunderstorm"
REASONING: "The activity is indoors, and indoor events are not affected by thunderstorms."
FINAL ANSWER: IS_COMPATIBLE

Example 2: 
  - Activity: "Palette & Palate: Art Meets Cooking Experience"
  - Weather: "sunny"
REASONING: "The activity is indoors, and weather does not affect it."
FINAL ANSWER: IS_COMPATIBLE

Example 3:
  - Activity: "Run Outside"
  - Weather: "thunderstorm"
REASONING: "The activity is outdoors, and thunderstorms make outdoor runs unsafe."
FINAL ANSWER: IS_INCOMPATIBLE

Example 4 (with backup option):
  - Activity: "Outdoor Concert"
  - Weather: "heavy rain"
  - Backup: "Indoor Acoustic Session"
REASONING: "The outdoor concert is incompatible with heavy rain. However, the backup is indoors, so the overall plan remains feasible."
FINAL ANSWER: IS_COMPATIBLE

Example 5:
  - Activity: "Picnic in the Park"
  - Weather: "thunderstorm"
  - Backup: None
`
""".strip()

def eval_activities_and_weather_are_compatible(
    vacation_info: VacationInfo, final_output: TravelPlan
):
    """Verifies that no outdoor-only activities are scheduled during inclement weather conditions.

    Args:
        vacation_info (dict): Contains the vacation details
        final_output (dict): Contains the itinerary details including daily activities and weather conditions

    Raises:
        AgentError: If any outdoor activities are scheduled during weather conditions that could ruin them
    """

    activities_that_are_incompatible = []

    for itinerary_day in final_output.itinerary_days:
        weather_condition = itinerary_day.weather.condition

        for activity_recommendation in itinerary_day.activity_recommendations:
            resp = do_chat_completion(
                messages=[
                    {
                        "role": "system",
                        "content": ACTIVITY_AND_WEATHER_ARE_COMPATIBLE_SYSTEM_PROMPT,
                    },
                    {
                        "role": "user",
                        "content": f"Activity: {activity_recommendation.activity.name}\nDescription: {activity_recommendation.activity.description}\nWeather Condition: {weather_condition}",
                    },
                ],
                client=client,
                # This is a high-frequency use case, so we use a fast and cheap model.
                model=MODEL,
            )
    


            if "IS_COMPATIBLE" in (resp or ""):
                is_compatible = True
            elif "IS_INCOMPATIBLE" in (resp or ""):
                is_compatible = False
            else:
                raise RuntimeError(
                    f"Unexpected response from the model: {resp}. Expected 'IS_COMPATIBLE' or 'IS_INCOMPATIBLE'."
                )

            if is_compatible:
                print(
                    f"✅ Activity {activity_recommendation.activity.name} (on {itinerary_day.date}) and weather '{weather_condition}' are compatible."
                )

            else:
                activities_that_are_incompatible.append(
                    activity_recommendation.activity.name
                )
                print(
                    f"❌ Activity {activity_recommendation.activity.name} (on {itinerary_day.date}) and weather '{weather_condition}' are incompatible."
                )

    if activities_that_are_incompatible:
        raise AgentError(
            f"Activities that may be ruined by inclement weather: {activities_that_are_incompatible}"
        )

In [ ]:
eval_results = get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=[
        eval_activities_and_weather_are_compatible
    ],
)
eval_results

In [ ]:
# perform all evalautions
ALL_EVAL_FUNCTIONS = [
    eval_start_end_dates_match,
    eval_total_cost_is_accurate,
    eval_itinerary_events_match_actual_events,
    eval_itinerary_satisfies_interests,
    eval_total_cost_is_within_budget,
    eval_activities_and_weather_are_compatible,
]

eval_results = get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=ALL_EVAL_FUNCTIONS,
)

eval_results.model_dump()

In [ ]:

def run_evals_tool(travel_plan: TravelPlan) -> dict:
    """Runs all evaluation tools on the provided travel plan and vacation info.

    Args:
        travel_plan (TravelPlan): The travel plan to evaluate.

    Returns:
        EvaluationResults: The results of the evaluations.
    """
    if isinstance(travel_plan, dict):
        travel_plan = TravelPlan.model_validate(travel_plan)

    resp = get_eval_results(
        vacation_info=vacation_info,
        final_output=travel_plan,
        eval_functions=ALL_EVAL_FUNCTIONS,
    )
    return {
        # Show the success status and any failures
        "success": resp.success,
        "failures": resp.failures,
    }




In [ ]:
run_evals_tool(travel_plan=travel_plan_1)

In [ ]:
ALL_TOOLS = [
    calculator_tool,
    get_activities_by_date_tool,
    run_evals_tool,
    final_answer_tool,
]
print(get_tool_descriptions_string(ALL_TOOLS))

In [ ]:
# The ItineraryRevisionAgent
TRAVELER_FEEDBACK = "I want to have at least two activities per day."


def eval_traveler_feedback_is_incorporated(
    vacation_info: VacationInfo, final_output: TravelPlan
):
    """Checks if the traveler's feedback was incorporated into the revised travel plan.

    Args:
        vacation_info (VacationInfo): The vacation information.
        final_output (TravelPlan): The revised travel plan.

    Raises:
        AgentError: If the traveler's feedback was not successfully incorporated.
    """

    agent = ChatAgent(
        system_prompt="""You are an expert in evaluating whether a travel plan incorporates traveler feedback.

    ## Output Format

    Respond using two sections (ANALYSIS AND FINAL OUTPUT) in the following format:

        ANALYSIS:
        * [step-by-step analysis]


        FINAL OUTPUT:
        [FULLY_INCORPORATED, PARTIALLY_INCORPORATED, NOT_INCORPORATED, or UNKNOWN]
        REASON: [reasoning for the final output]

    """,
        client=client,
        model=OpenAIModel.GPT_41,  # Use a powerful model for checking traveler feedback
    )

    resp = agent.chat(
        f"""Traveler Feedback: {TRAVELER_FEEDBACK}
    Revised Travel Plan: {final_output.model_dump_json()}
    """,
    )
    if "FINAL OUTPUT:" not in resp:
        raise RuntimeError(
            f"Unexpected response from the model: {resp}. Expected 'FINAL OUTPUT:'."
        )
    if "FULLY_INCORPORATED" not in resp:
        final_output = resp.split("FINAL OUTPUT:")[-1].strip()
        raise AgentError(
            f"Traveler feedback was not successfully incorporated into the revised travel plan. Response: {final_output}"
        )

ALL_EVAL_FUNCTIONS = [
    eval_start_end_dates_match,
    eval_total_cost_is_accurate,
    eval_itinerary_events_match_actual_events,
    eval_itinerary_satisfies_interests,
    eval_total_cost_is_within_budget,
    eval_activities_and_weather_are_compatible,
    eval_traveler_feedback_is_incorporated,  # Add this new evaluation
]

get_eval_results(
    vacation_info=vacation_info,
    final_output=travel_plan_1,
    eval_functions=ALL_EVAL_FUNCTIONS,
)

In [ ]:
ITINERARY_REVISION_AGENT_SYSTEM_PROMPT = f"""
you are a helpful agent and you should revise the itenerary to make it better.

## Task

- first, analyze the feedback and make proper revision to the travel itenrary
- use available tools like `get_activities_by_date_tool` or `calculate_tool` to gather information
- use `run_evals_tool` to evaluate the existing version of the travel plan
- Revise the plan incrementally, and after 2–3 iterations of evaluation, finalize it. 
- If evaluation keeps failing after 3 iterations, stop revising and choose the best available plan.
- Always end by calling `final_answer_tool` once you are confident OR after 3 iterations max.
- Never keep revising indefinitely.

## Available Tools

- `calculate_tool`: Evalautes a mathematical expression and returns a float value
   - args: 
        - input_expression (str): A string contianing the mathematical expression
   - returns: evalaute of the calculations
   - Example: 
      - calculator_tool("1 + 1")
      - result: 2
- `get_activities_by_date_tool`:
    - args:
        - date (str): date of the activity
        - city (str): name of the city where the acitivites are 
    - returns: list of acitivites for that date and city
- `run_evals_tool`
    - args: 
        - travel_plan (TravelPlan): the travel plan to evalaute
    - returns: evaluation results
- `final_answer_tool`
    - args: 
        - final_output (TravelPlan): the final travel plan
    - returns: the final travel plan

## Output Format

    THOUGHT:
    Understand the traveller plan and the feedback provided for that plan; make proper adjustments using the tools to apply the feedback; perform multiple iterations and run the evaluation tools again to get resolve any concerns from the feedback.
    

    ACTION:
    {{
        "tool_name": "[tool_name]",
        "arguments": {{
            "args1": "value1",
            ...
        }}
    }}

## Context

You have access to the current travel plan and traveler's feedback
Provide the final output in the following format:
FINAL_OUTPUT
```json
{travel_plan_json_schema}
```
travel plan: {travel_plan_1.model_dump_json()}
traveler feedback: {TRAVELER_FEEDBACK}

"""  # noqa


class ItineraryRevisionAgent(ChatAgent):
    system_prompt = ITINERARY_REVISION_AGENT_SYSTEM_PROMPT
    tools = ALL_TOOLS

    def get_observation_string(self, tool_call_obj) -> str:
        """Extracts the observation from the thought-action response."""

        if "tool_name" not in tool_call_obj:
            return "OBSERVATION: No tool name specified."

        if "arguments" not in tool_call_obj:
            return "OBSERVATION: No arguments specified."

        # If the arguments are not a dictionary, state the error
        if not isinstance(tool_call_obj["arguments"], dict):
            return f"OBSERVATION: Arguments should be a dictionary, got {type(tool_call_obj['arguments'])} instead."

        # If the tool name is not a string, state the error
        if not isinstance(tool_call_obj["tool_name"], str):
            return f"OBSERVATION: Tool name should be a string, got {type(tool_call_obj['tool_name'])} instead."

        tool_name = tool_call_obj["tool_name"]
        arguments = tool_call_obj["arguments"]

        tool_fn = None

        for tool in self.tools:
            if tool.__name__ == tool_name:
                tool_fn = tool
                break

        if tool_fn is None:
            return f"OBSERVATION: Unknown tool name '{tool_name}' in action string."

        try:
            tool_response = tool_fn(**arguments)
            return f"OBSERVATION: Tool {tool_name} called successfully with response: {tool_response}"
        except Exception as e:
            return f"OBSERVATION: Error occurred while calling tool {tool_name}: {e}"

    def run_react_cycle(
        self, original_travel_plan: TravelPlan, max_steps: int = 10, model: Optional[OpenAIModel] = None, client = None,
    ) -> TravelPlan:
        """Runs the ReAct cycle to revise the itinerary based on the evaluation results."""
        from json_repair import repair_json

        # Provide the original travel plan to revise
        self.add_message(
            role="user",
            content=f"Here is the itinerary for revision:\n{original_travel_plan.model_dump_json()}",
        )
        resp = None

        # Run the ReAct cycle for a maximum number of steps
        for step in range(max_steps):
            # Get the thought-action response from the agent
            resp = self.get_response(model=model, client=client) or ""

            # If there is no action, report it to the LLM and continue
            if "ACTION:" not in resp:
                self.add_message(role="user", content="No action found in response.")
                continue

            action_string = resp.split("ACTION:")[1].strip()

            # Parse the tool call JSON from the action string
            try:
                # Fix any JSON formatting issues. e.g. missing closing braces, etc.
                action_string = repair_json(action_string)
                tool_call_obj = json.loads(action_string)
            except json.JSONDecodeError:
                print(f"Invalid JSON in action string: {action_string}")
                self.add_message(
                    role="user",
                    content=f"Invalid JSON in action string: {action_string}",
                )
                continue

            tool_name = tool_call_obj.get("tool_name", None)

            # If the final answer tool is called, validate and return the final travel plan
            if tool_name == "final_answer_tool":
                try:
                    new_travel_plan = TravelPlan.model_validate(
                        tool_call_obj["arguments"].get("final_output", tool_call_obj["arguments"])
                    )
                    return new_travel_plan
                except Exception as e:
                    self.add_message(
                        role="user", content=f"Error validating final answer: {e}"
                    )
                    continue

            # For all other tools, execute the tool call and add the observation
            else:
                # Add the 
                observation_string = self.get_observation_string(
                    tool_call_obj=tool_call_obj
                )
                self.add_message(role="user", content=observation_string)

        raise RuntimeError(
            f"ReAct cycle did not complete within {max_steps} steps. Last response: {resp}"
        )

# Instantiate the Itinerary Revision Agent
itinerary_revision_agent = ItineraryRevisionAgent()

# Let's get a single THOUGHT/ACTION response back to check that the agent is working as expected.
resp = itinerary_revision_agent.chat(
    user_message=f"Here is the itinerary for revision: {travel_plan_1.model_dump_json(indent=2)}",
    add_to_messages=False,
    model=MODEL,
    client=client,
) or ""

print_in_box(resp, "Raw Response")
# Check for THOUGHT
if "THOUGHT:" in resp:
    print("✅ `THOUGHT:` found in raw the response, as expected.")
else:
    print("❌ Expected `THOUGHT:` in raw the response. Please check the system prompt (output format).")
# Check for ACTION
if "ACTION:" in resp:
    print("✅ `ACTION:` found in raw the response, as expected.")
else:
    print("❌ Expected `ACTION:` in raw the response. Please check the system prompt (output format).")
if "\"tool_name\"" in resp:
    print("✅ `\"tool_name\":` found in raw the response, as expected.")
else:
    print("❌ Expected `\"tool_name\":` in raw the response. Please check the system prompt (output format).")


In [ ]:
# Now let's run the ReAct cycle multiple times to get the revised itinerary.
# Note: If this takes more than a few minutes, then the agent may be stuck in a loop.
# Examine the traces to understand where it is failing and see if adjusting the system prompt helps.
# Since LLMs are stochastic, you will get different results each time you run this cell.
# No changes needed here.

itinerary_revision_agent = ItineraryRevisionAgent()
travel_plan_2 = itinerary_revision_agent.run_react_cycle(
    original_travel_plan=travel_plan_1, max_steps=15,
    model=MODEL,
    client=client,
)

print("✅ Revised itinerary generated successfully. Congratulations!")
